In [99]:
import pandas as pd
import spotipy
#from fycharts.SpotifyCharts import SpotifyCharts
from spotipy.oauth2 import SpotifyClientCredentials

In [6]:
df = pd.read_csv('global_2017-07-14--2017-07-21.csv')

In [7]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Despacito - Remix,Luis Fonsi,37572118,https://open.spotify.com/track/5CtI0qwDJkDQGwX...
2,2,Wild Thoughts (feat. Rihanna & Bryson Tiller),DJ Khaled,29984654,https://open.spotify.com/track/1OAh8uOEOvTDqkK...
3,3,Unforgettable,French Montana,25692670,https://open.spotify.com/track/3B54sVLJ402zGa6...
4,4,NaN,NaN,25442248,https://open.spotify.com/track/3RXkboS74UYzN14...
...,...,...,...,...,...
196,196,Perfect Strangers,Jonas Blue,3018660,https://open.spotify.com/track/5bZtRlMBU76vHuD...
197,197,Lean On (feat. MØ & DJ Snake),Major Lazer,3018465,https://open.spotify.com/track/4WjH9Bzt3kx7z8k...
198,198,Wonderwall - Remastered,Oasis,3012734,https://open.spotify.com/track/5qqabIl2vWzo9Ap...
199,199,Go Flex,Post Malone,3009022,https://open.spotify.com/track/5yuShbu70mtHXY0...


In [8]:
df.columns = df.iloc[0]

In [9]:
df = df.iloc[1:].reset_index()

In [10]:
df.URL.astype('str')

0      https://open.spotify.com/track/5CtI0qwDJkDQGwX...
1      https://open.spotify.com/track/1OAh8uOEOvTDqkK...
2      https://open.spotify.com/track/3B54sVLJ402zGa6...
3      https://open.spotify.com/track/3RXkboS74UYzN14...
4      https://open.spotify.com/track/3DXncPQOG4VBw3Q...
                             ...                        
195    https://open.spotify.com/track/5bZtRlMBU76vHuD...
196    https://open.spotify.com/track/4WjH9Bzt3kx7z8k...
197    https://open.spotify.com/track/5qqabIl2vWzo9Ap...
198    https://open.spotify.com/track/5yuShbu70mtHXY0...
199    https://open.spotify.com/track/0FuTx2s3YH1ppmt...
Name: URL, Length: 200, dtype: object

In [11]:
df.URL = df.URL.apply(lambda x : x[31:])

In [12]:
df.rename(columns={'URL':'ID'}, inplace=True)

In [13]:
df

,index,Position,Track Name,Artist,Streams,ID
0,1,1,Despacito - Remix,Luis Fonsi,37572118,5CtI0qwDJkDQGwXD1H1cLb
1,2,2,Wild Thoughts (feat. Rihanna & Bryson Tiller),DJ Khaled,29984654,1OAh8uOEOvTDqkKFsKksCi
2,3,3,Unforgettable,French Montana,25692670,3B54sVLJ402zGa6Xm4YGNe
3,4,4,NaN,NaN,25442248,3RXkboS74UYzN14xTqzPyY
4,5,5,"I'm the One (feat. Justin Bieber, Quavo, Chanc...",DJ Khaled,25431396,3DXncPQOG4VBw3QHh3S817
...,...,...,...,...,...,...
195,196,196,Perfect Strangers,Jonas Blue,3018660,5bZtRlMBU76vHuDOb1GM5u
196,197,197,Lean On (feat. MØ & DJ Snake),Major Lazer,3018465,4WjH9Bzt3kx7z8kl0awxh4
197,198,198,Wonderwall - Remastered,Oasis,3012734,5qqabIl2vWzo9ApSC317sa
198,199,199,Go Flex,Post Malone,3009022,5yuShbu70mtHXY0yLzCQLQ


In [14]:
credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
token = credentials.get_access_token()

spotify = spotipy.Spotify(auth=token)

features = spotify.audio_features(str(df['ID'].iloc[0]))


C:\Users\Mattia\anaconda3\envs\TrapGenerator\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


In [15]:
features

[{'danceability': 0.694,
  'energy': 0.815,
  'key': 2,
  'loudness': -4.328,
  'mode': 1,
  'speechiness': 0.12,
  'acousticness': 0.229,
  'instrumentalness': 0,
  'liveness': 0.0924,
  'valence': 0.813,
  'tempo': 88.931,
  'type': 'audio_features',
  'id': '5CtI0qwDJkDQGwXD1H1cLb',
  'uri': 'spotify:track:5CtI0qwDJkDQGwXD1H1cLb',
  'track_href': 'https://api.spotify.com/v1/tracks/5CtI0qwDJkDQGwXD1H1cLb',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5CtI0qwDJkDQGwXD1H1cLb',
  'duration_ms': 228827,
  'time_signature': 4}]

In [16]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        print(features[0]['danceability'])
        dataframe.append({'danceability' : features[0]['danceability'],
        'energy': features[0]['energy'],
        'key' : features[0]['key'],
        'loudness' : features[0]['loudness'],
        'mode' : features[0]['mode'],
        'speechiness' : features[0]['speechiness'],
        'acousticness' : features[0]['acousticness'],
        'instrumentalness' : features[0]['instrumentalness'],
        'liveness' : features[0]['liveness'],
        'valence' : features[0]['valence'],
        'tempo' : features[0]['tempo'],
        'time_signature' : features[0]['time_signature'],
        'duration_ms' : features[0]['duration_ms']}, ignore_index=True)
    return dataframe

In [17]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        dataframe.loc[i] = [ features[0]['id'],
        features[0]['danceability'],
        features[0]['energy'],
        features[0]['key'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['acousticness'],
        features[0]['instrumentalness'],
        features[0]['liveness'],
        features[0]['valence'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['duration_ms']]
    return dataframe

In [18]:
prova = extract_features(df, n_tracks=200)

In [19]:
prova

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,5CtI0qwDJkDQGwXD1H1cLb,0.694,0.815,2,-4.328,1,0.1200,0.229000,0,0.0924,0.813,88.931,4,228827
1,1OAh8uOEOvTDqkKFsKksCi,0.671,0.672,0,-3.094,0,0.0688,0.032900,0,0.1180,0.632,97.979,4,204173
2,3B54sVLJ402zGa6Xm4YGNe,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.0101,0.1040,0.733,97.985,4,233902
3,3RXkboS74UYzN14xTqzPyY,0.543,0.661,8,-5.855,0,0.0538,0.214000,0,0.2410,0.556,144.967,4,194978
4,3DXncPQOG4VBw3QHh3S817,0.609,0.668,7,-4.284,1,0.0367,0.055200,0,0.1670,0.811,80.924,4,288600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,5bZtRlMBU76vHuDOb1GM5u,0.739,0.833,1,-5.012,1,0.0463,0.350000,0,0.2660,0.699,117.990,4,195313
196,4WjH9Bzt3kx7z8kl0awxh4,0.723,0.809,7,-3.081,0,0.0625,0.003460,0.00123,0.5650,0.274,98.007,4,176561
197,5qqabIl2vWzo9ApSC317sa,0.378,0.896,2,-4.066,1,0.0407,0.000452,0,0.1940,0.469,174.302,4,258773
198,5yuShbu70mtHXY0yLzCQLQ,0.665,0.498,5,-8.185,1,0.0832,0.228000,0,0.0757,0.127,81.967,4,179613


In [20]:
output = df.join(prova.set_index('id'), on='ID')

In [64]:
output.to_csv('global_08-2017.csv')

In [28]:
output

test=output.iloc[9]
print(test)
print(type(test['Track Name']))
if type(test['Track Name']) != float and type(test['Artist']) != float:
    print('no empty')
else: 
    print('empty')

index                                   10
Position                                10
Track Name                             NaN
Artist                                 NaN
Streams                           20049646
ID                  4JAyIDXOqNM6qHuZML01uX
danceability                         0.699
energy                               0.614
key                                      2
loudness                             -4.57
mode                                     1
speechiness                         0.0521
acousticness                        0.0152
instrumentalness                  5.22e-06
liveness                            0.0734
valence                              0.275
tempo                               122.99
time_signature                           4
duration_ms                         186175
Name: 9, dtype: object
<class 'float'>
empty


## Testing iTunes API for genre labeling

In [23]:
import requests
import json
entry = 'starboy+the+weeknd'
a = 'https://itunes.apple.com/search?term='+entry+'.&limit=1'
b = requests.get(a).json()
print(b['results'][0]['primaryGenreName'])
print(json.dumps(b, indent=2))

R&B/Soul
{
  "resultCount": 1,
  "results": [
    {
      "wrapperType": "track",
      "kind": "song",
      "artistId": 479756766,
      "collectionId": 1440871397,
      "trackId": 1440871420,
      "artistName": "The Weeknd",
      "collectionName": "Starboy",
      "trackName": "Starboy (feat. Daft Punk)",
      "collectionCensoredName": "Starboy",
      "trackCensoredName": "Starboy (feat. Daft Punk)",
      "artistViewUrl": "https://music.apple.com/us/artist/the-weeknd/479756766?uo=4",
      "collectionViewUrl": "https://music.apple.com/us/album/starboy-feat-daft-punk/1440871397?i=1440871420&uo=4",
      "trackViewUrl": "https://music.apple.com/us/album/starboy-feat-daft-punk/1440871397?i=1440871420&uo=4",
      "previewUrl": "https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview128/v4/14/64/52/146452b1-d912-e0d4-d016-542de4bd8036/mzaf_4996492471208465628.plus.aac.p.m4a",
      "artworkUrl30": "https://is2-ssl.mzstatic.com/image/thumb/Music113/v4/9c/a2/d5/9ca2d550-4513-d

In [13]:
import os
path = os.getcwd()
file = 'global_2017-05-26--2017-06-02.csv'
print(path)

file_path = os.path.join(path,'global',file)
print(file_path)
df = pd.read_csv(file_path)

C:\Users\Mattia\Desktop\Polimi\Magistrale\Data Visualization\Data-and-Results-Visualization-Project
C:\Users\Mattia\Desktop\Polimi\Magistrale\Data Visualization\Data-and-Results-Visualization-Project\global\global_2017-05-26--2017-06-02.csv


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Mattia\\Desktop\\Polimi\\Magistrale\\Data Visualization\\Data-and-Results-Visualization-Project\\global\\global_2017-05-26--2017-06-02.csv'

In [36]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Fake Love,Drake,8618267,https://open.spotify.com/track/343YBumqHu19cGo...
2,2,Bad and Boujee (feat. Lil Uzi Vert),Migos,7946032,https://open.spotify.com/track/4Km5HrUvYTaSUfi...
3,3,Starboy,The Weeknd,7738517,https://open.spotify.com/track/5aAx2yezTd8zXrk...
4,4,Black Beatles,Rae Sremmurd,6209336,https://open.spotify.com/track/6fujklziTHa8uoM...
...,...,...,...,...,...
196,196,Dynamite (feat. Pretty Sister),Nause,1013332,https://open.spotify.com/track/2Ae5awwKvQpTBKQ...
197,197,Versace on the Floor,Bruno Mars,1011536,https://open.spotify.com/track/0kN8xEmgMW9mh7U...
198,198,No Type,Rae Sremmurd,1003471,https://open.spotify.com/track/4scpF6J5uMBvoh6...
199,199,Unsteady,X Ambassadors,1002415,https://open.spotify.com/track/7lGKEWMXVWWTt3X...


In [37]:
df.columns = df.iloc[0]

In [38]:
df = df.iloc[1:].reset_index()

In [39]:
df.URL.astype('str')

0      https://open.spotify.com/track/343YBumqHu19cGo...
1      https://open.spotify.com/track/4Km5HrUvYTaSUfi...
2      https://open.spotify.com/track/5aAx2yezTd8zXrk...
3      https://open.spotify.com/track/6fujklziTHa8uoM...
4      https://open.spotify.com/track/7BKLCZ1jbUBVqRi...
                             ...                        
195    https://open.spotify.com/track/2Ae5awwKvQpTBKQ...
196    https://open.spotify.com/track/0kN8xEmgMW9mh7U...
197    https://open.spotify.com/track/4scpF6J5uMBvoh6...
198    https://open.spotify.com/track/7lGKEWMXVWWTt3X...
199    https://open.spotify.com/track/4UknpXoLjlB7Mao...
Name: URL, Length: 200, dtype: object

In [40]:
df.URL = df.URL.apply(lambda x : x[31:])

In [41]:
df.rename(columns={'URL':'ID'}, inplace=True)

In [42]:
credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
token = credentials.get_access_token()

spotify = spotipy.Spotify(auth=token)

features = spotify.audio_features(str(df['ID'].iloc[0]))

C:\Users\Mattia\anaconda3\envs\TrapGenerator\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


In [43]:
features

[{'danceability': 0.928,
  'energy': 0.481,
  'key': 9,
  'loudness': -9.35,
  'mode': 0,
  'speechiness': 0.287,
  'acousticness': 0.105,
  'instrumentalness': 0,
  'liveness': 0.176,
  'valence': 0.613,
  'tempo': 134.007,
  'type': 'audio_features',
  'id': '343YBumqHu19cGoGARUTsd',
  'uri': 'spotify:track:343YBumqHu19cGoGARUTsd',
  'track_href': 'https://api.spotify.com/v1/tracks/343YBumqHu19cGoGARUTsd',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/343YBumqHu19cGoGARUTsd',
  'duration_ms': 210937,
  'time_signature': 4}]

In [44]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        print(features[0]['danceability'])
        dataframe.append({'danceability' : features[0]['danceability'],
        'energy': features[0]['energy'],
        'key' : features[0]['key'],
        'loudness' : features[0]['loudness'],
        'mode' : features[0]['mode'],
        'speechiness' : features[0]['speechiness'],
        'acousticness' : features[0]['acousticness'],
        'instrumentalness' : features[0]['instrumentalness'],
        'liveness' : features[0]['liveness'],
        'valence' : features[0]['valence'],
        'tempo' : features[0]['tempo'],
        'time_signature' : features[0]['time_signature'],
        'duration_ms' : features[0]['duration_ms']}, ignore_index=True)
    return dataframe

In [45]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        dataframe.loc[i] = [ features[0]['id'],
        features[0]['danceability'],
        features[0]['energy'],
        features[0]['key'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['acousticness'],
        features[0]['instrumentalness'],
        features[0]['liveness'],
        features[0]['valence'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['duration_ms']]
    return dataframe

In [46]:
prova = extract_features(df, n_tracks=200)
prova

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,343YBumqHu19cGoGARUTsd,0.928,0.481,9,-9.350,0,0.2870,0.1050,0,0.1760,0.613,134.007,4,210937
1,4Km5HrUvYTaSUfiSGPJeQR,0.927,0.665,11,-5.313,1,0.2440,0.0610,0,0.1230,0.175,127.076,4,343150
2,5aAx2yezTd8zXrkmtKl66Z,0.681,0.594,7,-7.028,1,0.2820,0.1650,3.49e-06,0.1340,0.535,186.054,4,230453
3,6fujklziTHa8uoM5OQSfIo,0.794,0.632,0,-6.163,1,0.0649,0.1420,0,0.1280,0.355,145.926,4,291893
4,7BKLCZ1jbUBVqRi2FVlTVw,0.748,0.524,8,-5.599,1,0.0338,0.4140,0,0.1110,0.661,95.010,4,244960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2Ae5awwKvQpTBKQHr1TYCg,0.811,0.801,9,-6.000,0,0.0802,0.0548,2.09e-05,0.0742,0.693,105.055,4,195429
196,0kN8xEmgMW9mh7UmDYHlJP,0.578,0.574,2,-6.209,1,0.0454,0.1960,0,0.0830,0.301,174.152,4,261240
197,4scpF6J5uMBvoh6sFB7EL1,0.891,0.486,6,-7.803,1,0.1590,0.0158,3.78e-06,0.0925,0.239,125.010,4,200080
198,7lGKEWMXVWWTt3X71Bv44I,0.389,0.665,0,-6.169,1,0.0644,0.1780,0.000732,0.1160,0.199,117.055,4,193547


In [47]:
output = df.join(prova.set_index('id'), on='ID')
output

,index,Position,Track Name,Artist,Streams,ID,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,1,1,Fake Love,Drake,8618267,343YBumqHu19cGoGARUTsd,0.928,0.481,9,-9.350,0,0.2870,0.1050,0,0.1760,0.613,134.007,4,210937
1,2,2,Bad and Boujee (feat. Lil Uzi Vert),Migos,7946032,4Km5HrUvYTaSUfiSGPJeQR,0.927,0.665,11,-5.313,1,0.2440,0.0610,0,0.1230,0.175,127.076,4,343150
2,3,3,Starboy,The Weeknd,7738517,5aAx2yezTd8zXrkmtKl66Z,0.681,0.594,7,-7.028,1,0.2820,0.1650,3.49e-06,0.1340,0.535,186.054,4,230453
3,4,4,Black Beatles,Rae Sremmurd,6209336,6fujklziTHa8uoM5OQSfIo,0.794,0.632,0,-6.163,1,0.0649,0.1420,0,0.1280,0.355,145.926,4,291893
4,5,5,Closer,The Chainsmokers,5813157,7BKLCZ1jbUBVqRi2FVlTVw,0.748,0.524,8,-5.599,1,0.0338,0.4140,0,0.1110,0.661,95.010,4,244960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,196,Dynamite (feat. Pretty Sister),Nause,1013332,2Ae5awwKvQpTBKQHr1TYCg,0.811,0.801,9,-6.000,0,0.0802,0.0548,2.09e-05,0.0742,0.693,105.055,4,195429
196,197,197,Versace on the Floor,Bruno Mars,1011536,0kN8xEmgMW9mh7UmDYHlJP,0.578,0.574,2,-6.209,1,0.0454,0.1960,0,0.0830,0.301,174.152,4,261240
197,198,198,No Type,Rae Sremmurd,1003471,4scpF6J5uMBvoh6sFB7EL1,0.891,0.486,6,-7.803,1,0.1590,0.0158,3.78e-06,0.0925,0.239,125.010,4,200080
198,199,199,Unsteady,X Ambassadors,1002415,7lGKEWMXVWWTt3X71Bv44I,0.389,0.665,0,-6.169,1,0.0644,0.1780,0.000732,0.1160,0.199,117.055,4,193547


In [48]:
output['genre']= "none"

In [49]:
output

,index,Position,Track Name,Artist,Streams,ID,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,genre
0,1,1,Fake Love,Drake,8618267,343YBumqHu19cGoGARUTsd,0.928,0.481,9,-9.350,0,0.2870,0.1050,0,0.1760,0.613,134.007,4,210937,none
1,2,2,Bad and Boujee (feat. Lil Uzi Vert),Migos,7946032,4Km5HrUvYTaSUfiSGPJeQR,0.927,0.665,11,-5.313,1,0.2440,0.0610,0,0.1230,0.175,127.076,4,343150,none
2,3,3,Starboy,The Weeknd,7738517,5aAx2yezTd8zXrkmtKl66Z,0.681,0.594,7,-7.028,1,0.2820,0.1650,3.49e-06,0.1340,0.535,186.054,4,230453,none
3,4,4,Black Beatles,Rae Sremmurd,6209336,6fujklziTHa8uoM5OQSfIo,0.794,0.632,0,-6.163,1,0.0649,0.1420,0,0.1280,0.355,145.926,4,291893,none
4,5,5,Closer,The Chainsmokers,5813157,7BKLCZ1jbUBVqRi2FVlTVw,0.748,0.524,8,-5.599,1,0.0338,0.4140,0,0.1110,0.661,95.010,4,244960,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,196,Dynamite (feat. Pretty Sister),Nause,1013332,2Ae5awwKvQpTBKQHr1TYCg,0.811,0.801,9,-6.000,0,0.0802,0.0548,2.09e-05,0.0742,0.693,105.055,4,195429,none
196,197,197,Versace on the Floor,Bruno Mars,1011536,0kN8xEmgMW9mh7UmDYHlJP,0.578,0.574,2,-6.209,1,0.0454,0.1960,0,0.0830,0.301,174.152,4,261240,none
197,198,198,No Type,Rae Sremmurd,1003471,4scpF6J5uMBvoh6sFB7EL1,0.891,0.486,6,-7.803,1,0.1590,0.0158,3.78e-06,0.0925,0.239,125.010,4,200080,none
198,199,199,Unsteady,X Ambassadors,1002415,7lGKEWMXVWWTt3X71Bv44I,0.389,0.665,0,-6.169,1,0.0644,0.1780,0.000732,0.1160,0.199,117.055,4,193547,none


In [70]:
import requests
import json
import time

for index, row in output.iterrows():
    artist = row["Artist"]
    song = row["Track Name"]
    print(song)
    song = song.split('(')[0]
    query_entry = str(artist)+" "+str(song)
    query_entry = query_entry.replace("with", "")
    query_entry=query_entry.replace(" ", "+")
    #query_entry = query_entry[0:60]
    query_entry = query_entry.lower()
    print(query_entry)
    a = 'https://itunes.apple.com/search?term='+query_entry+'&limit=1'
    b = requests.get(a).json()
    print(b['results'][0]['primaryGenreName'])
    #assign genre to song
    print(index)
    output.iloc[index, output.columns.get_loc('genre')]= b['results'][0]['primaryGenreName']
    time.sleep(2)
    #print(json.dumps(b, indent=2))

Despacito - Remix
luis+fonsi+despacito+-+remix
Pop
0
Wild Thoughts (feat. Rihanna & Bryson Tiller)
dj+khaled+wild+thoughts+
Hip-Hop/Rap
1
Unforgettable
french+montana+unforgettable
Hip-Hop/Rap
2
nan


AttributeError: 'float' object has no attribute 'split'

In [8]:
query_entry = 'xxxtentacion+carry+on'
a = 'https://itunes.apple.com/search?term='+query_entry+'&limit=1'
print(a)
test = requests.get(a)
print(test)
b = requests.get(a).json()
#print(b)
if b['resultCount'] == 1:
    print(b)
    print(b['results'][0]['primaryGenreName'])
else:
    print('no result')

https://itunes.apple.com/search?term=xxxtentacion+carry+on&limit=1
<Response [200]>
{'resultCount': 1, 'results': [{'wrapperType': 'track', 'kind': 'song', 'artistId': 1082533559, 'collectionId': 1274653476, 'trackId': 1274653785, 'artistName': 'XXXTENTACION', 'collectionName': '17', 'trackName': 'Carry On', 'collectionCensoredName': '17', 'trackCensoredName': 'Carry On', 'artistViewUrl': 'https://music.apple.com/us/artist/xxxtentacion/1082533559?uo=4', 'collectionViewUrl': 'https://music.apple.com/us/album/carry-on/1274653476?i=1274653785&uo=4', 'trackViewUrl': 'https://music.apple.com/us/album/carry-on/1274653476?i=1274653785&uo=4', 'previewUrl': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview128/v4/c6/ae/07/c6ae079d-f6f2-3f29-0433-237caf342e30/mzaf_1547727276898774776.plus.aac.p.m4a', 'artworkUrl30': 'https://is2-ssl.mzstatic.com/image/thumb/Music128/v4/09/c6/4d/09c64d53-e9e9-63b7-6241-1f4d270a3a54/source/30x30bb.jpg', 'artworkUrl60': 'https://is2-ssl.mzstatic.com/ima

In [153]:
output

,index,Position,Track Name,Artist,Streams,ID,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,genre
0,1,1,Fake Love,Drake,8618267,343YBumqHu19cGoGARUTsd,0.928,0.481,9,-9.350,0,0.2870,0.1050,0,0.1760,0.613,134.007,4,210937,Hip-Hop/Rap
1,2,2,Bad and Boujee (feat. Lil Uzi Vert),Migos,7946032,4Km5HrUvYTaSUfiSGPJeQR,0.927,0.665,11,-5.313,1,0.2440,0.0610,0,0.1230,0.175,127.076,4,343150,Hip-Hop
2,3,3,Starboy,The Weeknd,7738517,5aAx2yezTd8zXrkmtKl66Z,0.681,0.594,7,-7.028,1,0.2820,0.1650,3.49e-06,0.1340,0.535,186.054,4,230453,R&B/Soul
3,4,4,Black Beatles,Rae Sremmurd,6209336,6fujklziTHa8uoM5OQSfIo,0.794,0.632,0,-6.163,1,0.0649,0.1420,0,0.1280,0.355,145.926,4,291893,Hip-Hop/Rap
4,5,5,Closer,The Chainsmokers,5813157,7BKLCZ1jbUBVqRi2FVlTVw,0.748,0.524,8,-5.599,1,0.0338,0.4140,0,0.1110,0.661,95.010,4,244960,Dance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,196,Dynamite (feat. Pretty Sister),Nause,1013332,2Ae5awwKvQpTBKQHr1TYCg,0.811,0.801,9,-6.000,0,0.0802,0.0548,2.09e-05,0.0742,0.693,105.055,4,195429,Dance
196,197,197,Versace on the Floor,Bruno Mars,1011536,0kN8xEmgMW9mh7UmDYHlJP,0.578,0.574,2,-6.209,1,0.0454,0.1960,0,0.0830,0.301,174.152,4,261240,Pop
197,198,198,No Type,Rae Sremmurd,1003471,4scpF6J5uMBvoh6sFB7EL1,0.891,0.486,6,-7.803,1,0.1590,0.0158,3.78e-06,0.0925,0.239,125.010,4,200080,Hip-Hop/Rap
198,199,199,Unsteady,X Ambassadors,1002415,7lGKEWMXVWWTt3X71Bv44I,0.389,0.665,0,-6.169,1,0.0644,0.1780,0.000732,0.1160,0.199,117.055,4,193547,Alternative


# Automatizing the feature extraction process

In [11]:
import pandas as pd
import spotipy
#from fycharts.SpotifyCharts import SpotifyCharts
from spotipy.oauth2 import SpotifyClientCredentials
import os
import requests
import json
import time

In [12]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        print(features[0]['danceability'])
        dataframe.append({'danceability' : features[0]['danceability'],
        'energy': features[0]['energy'],
        'key' : features[0]['key'],
        'loudness' : features[0]['loudness'],
        'mode' : features[0]['mode'],
        'speechiness' : features[0]['speechiness'],
        'acousticness' : features[0]['acousticness'],
        'instrumentalness' : features[0]['instrumentalness'],
        'liveness' : features[0]['liveness'],
        'valence' : features[0]['valence'],
        'tempo' : features[0]['tempo'],
        'time_signature' : features[0]['time_signature'],
        'duration_ms' : features[0]['duration_ms']}, ignore_index=True)
    return dataframe

In [13]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        dataframe.loc[i] = [ features[0]['id'],
        features[0]['danceability'],
        features[0]['energy'],
        features[0]['key'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['acousticness'],
        features[0]['instrumentalness'],
        features[0]['liveness'],
        features[0]['valence'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['duration_ms']]
    return dataframe

In [14]:
#checking data integrity. Checks for missing fields in Artist name. Return the rows with missing values + the file name. 
#These data are missing because of copyright issues. They changed the author of the song/removed the song. 
import numpy as np

from IPython.display import display, HTML

path = os.getcwd()
file_folder = os.path.join(path,'global')

for csv_file in os.listdir(file_folder):
    file = os.path.join(file_folder,csv_file)
    #print(csv_file)
    df = pd.read_csv(file)
    #display(df)
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index()
    df.URL.astype('str')
    df.URL = df.URL.apply(lambda x : x[31:])
    df.rename(columns={'URL':'ID'}, inplace=True)
    
    res = df.loc[df['Artist'].isnull(), 'index'].tolist()
    if len(res) != 0:
        print(res)
        print(csv_file)

    
    
 

[4, 10, 30, 40]
global_2017-07-14--2017-07-21.csv
[32]
global_2017-11-03--2017-11-10.csv


In [17]:
path = os.getcwd()
file_folder = os.path.join(path,'global 2')
save_folder_path = os.path.join(path, "global_features")
#os.mkdir(save_folder_path)

for csv_file in os.listdir(file_folder): 
    print(csv_file)
    file = os.path.join(file_folder,csv_file)
    df = pd.read_csv(file)
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index()
    df.URL.astype('str')
    df.URL = df.URL.apply(lambda x : x[31:])
    df.rename(columns={'URL':'ID'}, inplace=True)
    credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
    token = credentials.get_access_token()
    spotify = spotipy.Spotify(auth=token)
    features = spotify.audio_features(str(df['ID'].iloc[0]))
    prova = extract_features(df, n_tracks=200)
    output = df.join(prova.set_index('id'), on='ID')
    output['genre']= "none"
    for index, row in output.iterrows():
        artist = row["Artist"]
        song = row["Track Name"]
        if type(artist) != float and type(song) != float:
            song = song.split('(')[0]
            query_entry = str(artist)+" "+str(song)
            query_entry = query_entry.replace("with", "")
            query_entry= query_entry.replace('Remastered', "")
            query_entry= query_entry.replace('Version', "")

            query_entry = query_entry.replace("'", "")
            query_entry = query_entry.replace('"', "")

            query_entry=query_entry.replace(" ", "+")

            #query_entry = query_entry[0:60]
            query_entry = query_entry.lower()
            print(query_entry)
            a = 'https://itunes.apple.com/search?term='+query_entry+'&limit=1'
            test = requests.get(a)
            print(test)
            b = test.json()
            if b['resultCount'] == 1:

                print(b['results'][0]['primaryGenreName'])
                #assign genre to song
                print(index)
                output.iloc[index, output.columns.get_loc('genre')]= b['results'][0]['primaryGenreName']
            time.sleep(1)
            #print(json.dumps(b, indent=2))
    output.to_csv(os.path.join(save_folder_path,csv_file))




global_2018-07-20--2018-07-27.csv


C:\Users\Mattia\anaconda3\envs\TrapGenerator\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  app.launch_new_instance()


drake+in+my+feelings
<Response [200]>
Hip-Hop/Rap
0
cardi+b+i+like+it
<Response [200]>
Hip-Hop/Rap
1
maroon+5+girls+like+you+
<Response [200]>
Pop
2
ariana+grande+god+is+a+woman
<Response [200]>
Pop
3
juice+wrld+lucid+dreams
<Response [200]>
Hip-Hop/Rap
4
tyga+taste+
<Response [200]>
Hip-Hop/Rap
5
xxxtentacion+sad!
<Response [200]>
Pop
6
clean+bandit+solo+
<Response [200]>
Dance
7
calvin+harris+one+kiss+
<Response [200]>
Dance
8
post+malone+better+now
<Response [200]>
Hip-Hop/Rap
9
tiësto+jackie+chan
<Response [200]>
Dance
10
jonas+blue+rise
<Response [200]>
Dance
11
drake+don’t+matter+to+me+
<Response [200]>
Hip-Hop/Rap
12
5+seconds+of+summer+youngblood
<Response [200]>
Pop
13
6ix9ine+fefe+
<Response [200]>
Hip-Hop/Rap
14
drake+nonstop
<Response [200]>
Hip-Hop/Rap
15
nio+garcia+te+boté+-+remix
<Response [200]>
Urbano latino
16
xxxtentacion+moonlight
<Response [200]>
Pop
17
drake+gods+plan
<Response [200]>
Hip-Hop/Rap
18
benny+blanco+eastside+
<Response [200]>
Pop
19
martin+garrix+ocea

twenty+one+pilots+jumpsuit
<Response [200]>
Alternative
157
oasis+wonderwall+-+
<Response [200]>
Alternative
158
travis+scott+goosebumps
<Response [200]>
Hip-Hop/Rap
159
piso+21+déjala+que+vuelva+
<Response [200]>
Urbano latino
160
justin+quiles+no+quiero+amarte+
<Response [200]>
Urbano latino
161
troye+sivan+dance+to+this+
<Response [200]>
Pop
162
the+chainsmokers+somebody
<Response [200]>
Dance
163
maroon+5+what+lovers+do+
<Response [200]>
Pop
164
marshmello+silence
<Response [200]>
Dance
165
danny+ocean+dembow
<Response [200]>
Latino
166
camila+cabello+never+be+the+same
<Response [200]>
Pop
167
david+guetta+like+i+do
<Response [200]>
Dance
168
lil+pump+drug+addicts
<Response [200]>
Hip-Hop/Rap
169
axwell+/\+ingrosso+more+than+you+know
<Response [200]>
Dance
170
imagine+dragons+whatever+it+takes
<Response [200]>
Alternative
171
rudimental+let+me+live+
<Response [200]>
Dance
172
ed+sheeran+thinking+out+loud
<Response [200]>
Pop
173
khalid+young+dumb+&+broke
<Response [200]>
R&B/Soul
1

sam+smith+too+good+at+goodbyes
<Response [200]>
Pop
112
the+weeknd+call+out+my+name
<Response [200]>
R&B/Soul
113
tropijazz+síguelo+bailando
<Response [200]>
sia+audio
<Response [200]>
Pop
115
kendrick+lamar+all+the+stars+
<Response [200]>
Hip-Hop/Rap
116
liam+payne+familiar
<Response [200]>
Pop
117
kygo+remind+me+to+forget
<Response [200]>
Dance
118
kanye+west+all+mine
<Response [200]>
Hip-Hop/Rap
119
daddy+yankee+dura
<Response [200]>
Urbano latino
120
maluma+el+préstamo
<Response [200]>
Urbano latino
121
migos+walk+it+talk+it
<Response [200]>
Hip-Hop/Rap
122
piso+21+te+amo
<Response [200]>
Urbano latino
123
blackpink+ddu-du+ddu-du+-+kr+ver.
<Response [200]>
Pop
124
the+chainsmokers+closer
<Response [200]>
Dance
125
jason+mraz+im+yours
<Response [200]>
Pop
126
xxxtentacion+hope
<Response [200]>
Pop
127
leslie+grace+duro+y+suave
<Response [200]>
Pop Latino
128
kendrick+lamar+humble.
<Response [200]>
Hip-Hop/Rap
129
offset+ric+flair+drip+
<Response [200]>
Hip-Hop/Rap
130
avicii+wake+me

...

tyga+taste+
<Response [200]>
Hip-Hop/Rap
124
capital+bra+cherry+lady
<Response [200]>
Hip-Hop/Rap
125
khalid+self
<Response [200]>
R&B/Soul
126
calboy+envy+me
<Response [200]>
Hip-Hop/Rap
127
xxxtentacion+jocelyn+flores
<Response [200]>
Hip-Hop/Rap
128
lukas+graham+love+someone
<Response [200]>
Pop
129
drake+gods+plan
<Response [200]>
Hip-Hop/Rap
130
gucci+mane+wake+up+in+the+sky
<Response [200]>
Hip-Hop/Rap
131
sofia+reyes+r.i.p.+
<Response [200]>
Pop Latino
132
bebe+rexha+last+hurrah
<Response [200]>
Pop
133
maren+morris+the+bones
<Response [200]>
Country
134
ariana+grande+no+tears+left+to+cry
<Response [200]>
Pop
135
calvin+harris+one+kiss+
<Response [200]>
Dance
136
billie+eilish+bellyache
<Response [200]>
Alternative
137
khalid+hundred
<Response [200]>
R&B/Soul
138
mark+ronson+nothing+breaks+like+a+heart+
<Response [200]>
Pop
139
khalid+intro
<Response [200]>
R&B/Soul
140
sam+smith+too+good+at+goodbyes
<Response [200]>
Pop
141
shawn+mendes+lost+in+japan
<Response [200]>
Pop
142
bi

anuel+aa+amanece
<Response [200]>
Urbano latino
82
imagine+dragons+bad+liar
<Response [200]>
Alternative
83
anuel+aa+ella+quiere+beber+-+remix
<Response [200]>
Urbano latino
84
juice+wrld+lucid+dreams
<Response [200]>
Hip-Hop/Rap
85
ynw+melly+mixed+personalities+
<Response [200]>
Hip-Hop/Rap
86
capital+bra+rolex
<Response [200]>
Hip-Hop/Rap
87
ellie+goulding+close+to+me+
<Response [200]>
Pop
88
manuel+turizo+esclavo+de+tus+besos
<Response [200]>
Urbano latino
89
xxxtentacion+sad!
<Response [200]>
Pop
90
ed+sheeran+shape+of+you
<Response [200]>
Pop
91
nipsey+hussle+racks+in+the+middle+
<Response [200]>
Hip-Hop/Rap
92
ellie+goulding+sixteen
<Response [200]>
Pop
93
21+savage+a+lot
<Response [200]>
Hip-Hop/Rap
94
billie+eilish+i+love+you
<Response [200]>
Alternative
95
lil+baby+drip+too+hard+
<Response [200]>
Hip-Hop/Rap
96
tom+walker+just+you+and+i
<Response [200]>
Singer/Songwriter
97
blueface+thotiana
<Response [200]>
Hip-Hop/Rap
98
billie+eilish+ilomilo
<Response [200]>
Alternative
99


anuel+aa+secreto
<Response [200]>
Urbano latino
35
billie+eilish+lovely
<Response [200]>
Pop
36
lil+uzi+vert+sanguine+paradise
<Response [200]>
Hip-Hop/Rap
37
meek+mill+going+bad+
<Response [200]>
Hip-Hop/Rap
38
panic!+at+the+disco+high+hopes
<Response [200]>
Alternative
39
alan+walker+on+my+way
<Response [200]>
Dance
40
ava+max+so+am+i
<Response [200]>
Pop
41
paulo+londra+tal+vez
<Response [200]>
Latino
42
rosalía+con+altura
<Response [200]>
Pop
43
lil+dicky+earth
<Response [200]>
Pop
44
ynw+melly+murder+on+my+mind
<Response [200]>
Hip-Hop/Rap
45
billie+eilish+you+should+see+me+in+a+crown
<Response [200]>
Alternative
46
calvin+harris+giant+
<Response [200]>
Dance
47
khalid+better
<Response [200]>
R&B/Soul
48
dj+snake+taki+taki+
<Response [200]>
Dance
49
billie+eilish+all+the+good+girls+go+to+hell
<Response [200]>
Alternative
50
lewis+capaldi+someone+you+loved
<Response [200]>
Alternative
51
queen+bohemian+rhapsody+-+2011+mix
<Response [200]>
Rock
52
benny+blanco+eastside+
<Response [2

post+malone+wow.+-+remix
<Response [200]>
Pop
189
guaynaa+rebota
<Response [200]>
Urbano latino
190
schoolboy+q+chopstix+
<Response [200]>
Hip-Hop/Rap
191
bebe+rexha+last+hurrah
<Response [200]>
Pop
192
ed+sheeran+thinking+out+loud
<Response [200]>
Pop
193
zara+larsson+dont+worry+bout+me
<Response [200]>
Pop
194
drake+nonstop
<Response [200]>
Hip-Hop/Rap
195
jonas+blue+rise
<Response [200]>
Dance
196
sia+no+new+friends+
<Response [200]>
Pop
197
dua+lipa+kiss+and+make+up
<Response [200]>
Pop
198
nio+garcia+te+boté+-+remix
<Response [200]>
Urbano latino
199
global_2019-04-26--2019-05-03.csv
billie+eilish+bad+guy
<Response [200]>
Alternative
0
taylor+swift+me!+
<Response [200]>
Country
1
lil+nas+x+old+town+road+
<Response [200]>
Country
2
lil+nas+x+old+town+road
<Response [200]>
Country
3
avicii+sos
<Response [200]>
Dance
4
daddy+yankee+con+calma
<Response [200]>
Urbano latino
5
post+malone+sunflower+-+spider-man:+into+the+spider-verse
<Response [200]>
Hip-Hop/Rap
6
ariana+grande+7+rings


ConnectionError: HTTPSConnectionPool(host='itunes.apple.com', port=443): Max retries exceeded with url: /search?term=kodak+black+zeze+&limit=1 (Caused by NewConnectionError("<urllib3.connection.VerifiedHTTPSConnection object at 0x0000023648528248>: Failed to establish a new connection: [WinError 10060] Impossibile stabilire la connessione. Risposta non corretta della parte connessa dopo l'intervallo di tempo oppure mancata risposta dall'host collegato"))

In [57]:
string = 'sia+helium+-+from+"fifty+shades+darker+'
string=string.replace('"', "")
print(string)


sia+helium+-+from+fifty+shades+darker+
